## 复习

- django进行restful开发
- pyjwt
    - encode()
    - decode()
- 前端vue组件项目

# 今天知识点

- nginx基本配置
- nginx负载均衡

# 安装

- 目录/usr/local/nginx
- conf===>配置文件
- logs===>日志文件
- sbin/nginx===>主程序
- 优点
- 缺点

- 启动：sudo nginx
- 停止：sudo nginx -s stop
- 平滑重启：sudo nginx -s reload
- 查看版本：nginx -V

# nginx基本配置

- 全局配置
- http段配置
- server段配置
    - listen 端口
    - server_name 域名
    - root 项目根目录
    - index 默认文件名称
    - return 状态码
- location配置：路由规则
    - 优先级：= ~ 空
    - 特殊符号：
        - *不区分大小写
        - ^表示开始
        - $表示结束
        - ！表示非
        - @表示自定义处理程序
    - 命令
        - root
        - alias
        - retry_files尝试
        - index
        - return
    - 区分：root与alias
        - root：将请求路径与root指定路径拼接获取文件
        - alias：将请求路径中匹配字符去掉，再与alias指定路径拼接，获取文件
        - 用途：alias可以用于隐藏服务器真实目录名称

# nginx负载均衡

- 正向代理、反向代理
- 负载均衡===》nginx
- 代码实现

```
upstream 集群名称{
    server 服务器1地址;
    server 服务器2地址;
    ....
}

server {
    。。。。
    location / {
        proxy_pass http://集群名称;
    }
}
```

# 总结

### 重要知识点

- http静态文件服务器配置
- 反向代码转发配置

### 作业

- 练习基本配置、负载均衡配置
- 扩展阅读：[Nginx服务器之负载均衡策略（6种）](https://www.cnblogs.com/1214804270hacker/p/9325150.html)、[负载均衡调度算法大全](https://blog.csdn.net/weiyuefei/article/details/50513667)

### 大纲要求

- 能够知道ngixn的几个配置段

```
全局段
http{
    ....
    server {
        listen ;
        server_name ;
        location ...{
            ....
        }
    }
}
```

- 知道 反向代理和正向代理区别，重点反向代理

```
正向代理：在公司内部与互联网之间的一台代理服务器
反向代理：在服务器集群与互联网之间的一台代理服务器
```

- 知道 负载均衡的集中调度算法

```
轮询
加权轮询
ip_hash
least_conn
fair
url_hash
```


# 布署美多商城项目

- 主要分为4步
    - 前台静态文件
    - 后台静态文件
    - django项目
    - nginx转发

### 域名规则

- 见如下表格

域名 | 用途
---|---
www.meiduo.site | 前台
admin.meiduo.site:81 | 后台前端
api.meiduo.site:82 | 后台后端
image.meiduo.site:8888 | fastdfs中的图片

- 根据表格域名，编辑/etc/hosts文件

```
127.0.0.1   www.meiduo.site
127.0.0.1   image.meiduo.site
127.0.0.1   admin.meiduo.site
127.0.0.1   api.meiduo.site
```

### 前台静态文件

- 修改/static/js/host.js文件

```
// var host = 'http://127.0.0.1:8000';
// var host = 'http://www.meiduo.site:8000';
var host = 'http://www.meiduo.site';
```

- 指定静态文件目录

```
在dev.py中配置项
STATIC_ROOT = '磁盘目录'
```

- 收集静态文件

```
python manage.py collectstatic
```

- 配置nginx，访问静态文件

```
    server {
        listen       80;
        server_name  www.meiduo.site;
        location /static {
            root 静态文件目录;
        }
        location /detail {
            root 静态文件目录/static;
         }
        location = / {
            root    静态文件目录/static;
            index   index.html;
        }
        location = /index.html {
            root    静态文件目录/static;
        }
    }
```

- 在浏览器中输入域名www.meiduo.site，可以访问网站静态资源

### 后台静态文件

- 修改/src/components/constant.js文件

```
let cons = {
	// apis:'http://127.0.0.1:8000/meiduo_admin' // 远程地址
	apis:'http://api.meiduo.site:82/meiduo_admin'
}

export default cons
```

- 编译

```
npm run build
```

- 编译后会创建dist目录，这个目录下的内容可以交给运维人员使用
- 配置nginx，访问后台静态文件

```
    server {
        listen 81;
        server_name admin.meiduo.site;
        location /{
            root 上一步中的dist目录;
            index index.html;
        }
    }
```

- 在浏览器中输入域名admin.meiduo.site，可以访问网站静态资源

### django项目

- 包uwsgi：高效稳定运行python web程序

```
pip install uwsgi
```

- 在setttings目录下新建prod.py，复制dev.py的内容，修改代码

```
DEBUG = False

ALLOWED_HOSTS = [
    'www.meiduo.site',
    'api.meiduo.site',
]

# CORS  添加跨域白名单
CORS_ORIGIN_WHITELIST = (
    'admin.meiduo.site:81',
)
```

- 修改项目中的wsgi.py文件

```
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "meiduo_mall.settings.prod")
```

- 在项目目录下新建uwsgi.ini，内容如下

```
[uwsgi]
# 使用Nginx连接时使用，Django程序所在服务器地址
socket=127.0.0.1:8001
# 项目目录
chdir=项目路径/meiduo_project/meiduo_mall
# 项目中wsgi.py文件的目录，相对于项目目录
wsgi-file=meiduo_mall/wsgi.py
# 进程数
processes=4
# 线程数
threads=2
# uwsgi服务器的角色
master=True
# 存放进程编号的文件
pidfile=logs/uwsgi.pid
# 日志文件
daemonize=logs/uwsgi.log
```

- 控制uwsgi的命令

```
启动：uwsgi --ini uwsgi.ini
关闭：uwsgi --stop logs/uwsgi.pid
```

### nginx转发

- nginx接收到请求，将请求转发给uwsgi
- 配置nginx

```
    upstream meiduo{
        server 127.0.0.1:8001;
    }
    
    server {
        listen 80;
        server_name www.meiduo.site;
        
        ...原有配置不变，加入下面的代码

        location /{
            uwsgi_pass meiduo;
            include uwsgi_params;
        }
    }
    
    server {
        listen 82;
        server_name api.meiduo.site;
        location /{
            include uwsgi_params;
            uwsgi_pass meiduo;
        }
    }
```
